#Make a smaller notebook just to try and get a handle on how to merge two files...
24 August 2015

In [74]:
import pandas as pd
import urllib2
from bs4 import BeautifulSoup
import re
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import palettable as pal

%matplotlib inline

In [75]:
#mtabFile = 'RImetabolites_unique.2015.08.19.csv' #this was the file with the mtabs that only have one isomer in KEGG
mtabFile = 'RImetabolites_isomers.2015.08.21.csv' #first column is RInumber

In [76]:
CO_RawData=pd.read_csv(mtabFile, index_col='RInumber')
#can immediately see the multiple iterations of a CO number right at the top five rows. 
#The compound labeled as C00079 shows up at least three times bc found at 
#different retention times. Need to track this since cNumber is not unique
##BUT, only matters in the Kmeans clustering, does not matter in the data gathering from KEGG
CO_RawData.head(n=5)

,cNumber,S1,S2,S3,S4,S5
RInumber,,,,,,
RI1,C00047,0.000841,0.001725,0.001201,0.000771,0.000604
RI2,C00078,0.005920,0.007708,0.003950,0.010562,0.027481
RI3,C00079,0.000229,0.000310,0.000320,0.000169,0.000356
RI4,C00079,0.002722,0.004054,0.002479,0.002422,0.001029
RI5,C00079,0.031454,0.045236,0.017831,0.040436,0.086708


So, not sure what the best way is to organize this...dive in and see what happens. First, try pulling out the RInumber into its own matrix. Then most of the code should be the same as before, but will still have a unique number to track at the end.

In [242]:
CO_RawData_all = pd.read_csv('RImetabolites_isomers.2015.08.21.csv',index_col = 'RInumber')

In [243]:
df = CO_RawData_all

In [244]:
df.columns

Index([u'cNumber', u'S1', u'S2', u'S3', u'S4', u'S5'], dtype='object')

In [245]:
df.index

Index([u'RI1', u'RI2', u'RI3', u'RI4', u'RI5', u'RI6', u'RI7', u'RI8', u'RI9', u'RI10', u'RI11', u'RI12', u'RI13', u'RI14', u'RI15', u'RI16', u'RI17', u'RI18', u'RI19', u'RI20', u'RI21', u'RI22', u'RI23', u'RI24', u'RI25', u'RI26', u'RI27', u'RI28', u'RI29', u'RI30', u'RI31', u'RI32', u'RI33', u'RI34', u'RI35', u'RI36', u'RI37', u'RI38', u'RI39', u'RI40', u'RI41', u'RI42', u'RI43', u'RI44', u'RI45', u'RI46', u'RI47', u'RI48', u'RI49', u'RI50', u'RI51', u'RI52', u'RI53', u'RI54', u'RI55', u'RI56', u'RI57', u'RI58', u'RI59', u'RI60', u'RI61', u'RI62', u'RI63', u'RI64', u'RI65', u'RI66', u'RI67', u'RI68', u'RI69', u'RI70', u'RI71', u'RI72', u'RI73', u'RI74', u'RI75', u'RI76', u'RI77', u'RI78', u'RI79', u'RI80', u'RI81', u'RI82', u'RI83', u'RI84', u'RI85', u'RI86', u'RI87', u'RI88', u'RI89', u'RI90', u'RI91', u'RI92', u'RI93', u'RI94', u'RI95', u'RI96', u'RI97', u'RI98', u'RI99', u'RI100', ...], dtype='object')

In [249]:
df.ix[0,'cNumber']

'C00047'

In [248]:
df.head(5)

,cNumber,S1,S2,S3,S4,S5
RInumber,,,,,,
RI1,C00047,0.000841,0.001725,0.001201,0.000771,0.000604
RI2,C00078,0.005920,0.007708,0.003950,0.010562,0.027481
RI3,C00079,0.000229,0.000310,0.000320,0.000169,0.000356
RI4,C00079,0.002722,0.004054,0.002479,0.002422,0.001029
RI5,C00079,0.031454,0.045236,0.017831,0.040436,0.086708


In [236]:
'S1' in CO_RawData_all.keys()

True

In [237]:
CO_RawData_all.head(5)

,cNumber,S1,S2,S3,S4,S5
RInumber,,,,,,
RI1,C00047,0.000841,0.001725,0.001201,0.000771,0.000604
RI2,C00078,0.005920,0.007708,0.003950,0.010562,0.027481
RI3,C00079,0.000229,0.000310,0.000320,0.000169,0.000356
RI4,C00079,0.002722,0.004054,0.002479,0.002422,0.001029
RI5,C00079,0.031454,0.045236,0.017831,0.040436,0.086708


In [80]:
#get rid of the cNumber...? or make a new matrix without it? probably best to make a new matrix
#without it
somelist = ['S1','S2','S3','S4','S5']
CO_RawData = CO_RawData_all[somelist]
CO_RawData.head(5)

,S1,S2,S3,S4,S5
RInumber,,,,,
RI1,0.000841,0.001725,0.001201,0.000771,0.000604
RI2,0.005920,0.007708,0.003950,0.010562,0.027481
RI3,0.000229,0.000310,0.000320,0.000169,0.000356
RI4,0.002722,0.004054,0.002479,0.002422,0.001029
RI5,0.031454,0.045236,0.017831,0.040436,0.086708


In [81]:
#now, keep the rows where the CO number has a matching KO number...but remember, still have duplicate CNumbers
RIandCO = CO_RawData_all['cNumber']

In [82]:
RIandCO.head(5)

RInumber
RI1         C00047
RI2         C00078
RI3         C00079
RI4         C00079
RI5         C00079
Name: cNumber, dtype: object

In [83]:
type(RIandCO)

pandas.core.series.Series

In [84]:
#while I would think this finds the index...keys here appears to be the columns with data?
'cNumber' in CO_RawData_all.keys()

True

In [85]:
'RInumber' in CO_RawData_all.keys()

False

In [86]:
list(CO_RawData_all.columns.values)

['cNumber', 'S1', 'S2', 'S3', 'S4', 'S5']

In [87]:
RIandCO[RIandCO == 'C00047']

RInumber
RI1         C00047
Name: cNumber, dtype: object

In [88]:
RIandCO[0]==AllCO

False

In [89]:
import cPickle as cpk
WorkingFile = cpk.load(open('running_Script.pickle','r'))

In [90]:
def SplitCODict(WorkingFile):
    CO_withoutKO={}
    CO_withKO={}
    for CO in WorkingFile.keys():

        if WorkingFile[CO]['Related KO']==[]:
            CO_withoutKO[CO]=WorkingFile[CO]
        else:
            CO_withKO[CO]=WorkingFile[CO]
    return CO_withoutKO, CO_withKO

CO_withoutKO, CO_withKO=SplitCODict(WorkingFile)
print 'There are', len(CO_withKO), 'COs with an associated KO.', len(CO_withoutKO), 'are not associated with a KO.'



There are 303 COs with an associated KO. 1036 are not associated with a KO.


In [91]:
AllKO=[]
AllCO=[]
for key in CO_withKO:
    AllKO.append(CO_withKO[key]['Related KO'])
    AllCO.append(CO_withKO[key]['Related CO'])
AllKO=list(set([item for sublist in AllKO for item in sublist]))
AllCO=list(set([item for sublist in AllCO for item in sublist]))

# CO_limited_Norm2Mean=CO_Norm2Mean.loc[AllCO].dropna()

In [180]:
list(CO_RawData_all.columns.values)

['cNumber', 'S1', 'S2', 'S3', 'S4', 'S5', 'new']

In [95]:
CO_RawData_all.head(5)

,cNumber,S1,S2,S3,S4,S5
RInumber,,,,,,
RI1,C00047,0.000841,0.001725,0.001201,0.000771,0.000604
RI2,C00078,0.005920,0.007708,0.003950,0.010562,0.027481
RI3,C00079,0.000229,0.000310,0.000320,0.000169,0.000356
RI4,C00079,0.002722,0.004054,0.002479,0.002422,0.001029
RI5,C00079,0.031454,0.045236,0.017831,0.040436,0.086708


In [42]:
m = CO_RawData_all['cNumber'][0]
m

'C00047'

In [114]:
m in AllCO

True

In [115]:
type(AllCO)

list

In [187]:
CO_RawData_all.keys()

Index([u'cNumber', u'S1', u'S2', u'S3', u'S4', u'S5', u'new'], dtype='object')

In [297]:
df.head(5)

,cNumber,S1,S2,S3,S4,S5,new
RInumber,,,,,,,
RI1,C00047,0.000841,0.001725,0.001201,0.000771,0.000604,True
RI2,C00078,0.005920,0.007708,0.003950,0.010562,0.027481,True
RI3,C00079,0.000229,0.000310,0.000320,0.000169,0.000356,True
RI4,C00079,0.002722,0.004054,0.002479,0.002422,0.001029,True
RI5,C00079,0.031454,0.045236,0.017831,0.040436,0.086708,True


In [298]:
df.iloc[1].loc['cNumber']

'C00078'

In [301]:
# df.ix[0,'cNumber'] #this will allow me to mix integers with labels
#more current way apparently uses iloc and loc, to use integers and labels respectively
# this would be df.iloc[0].loc['cNumber] {can't get that to work in the if statement}
#go through CO_RawData_all one row at a time (inefficient for sure, but I understand what is happening)
#make a new column in CO_RawData_all
CO_RawData_all['new'] = ""

for idx in range(0,len(CO_RawData_all)):
# for idx in range(0):
    fc = CO_RawData_all.ix[idx,'cNumber']
    if fc in AllCO:
        CO_RawData_all.ix[idx,'new'] = True
    else:
        CO_RawData_all.ix[idx,'new'] = False       

In [302]:
fc

'C18316'

In [304]:
CO_RawData_all.to_csv('check2.csv')

In [281]:
'C00525' in AllCO

False

In [51]:
CO_RawData_all.shape

(1614, 6)

In [52]:
type(AllCO)

list